In [1]:
import sys
dirname = '/Users/muhammaddaffarobani/Documents/personal_research/smt'
if dirname not in sys.path:
    sys.path.append(dirname)
    
from smt.utils.design_space import (
    DesignSpace,
    FloatVariable,
    CategoricalVariable,
)
from smt.applications.mixed_integer import (
    MixedIntegerKrigingModel,
)
from smt.surrogate_models import (
    KPLS,
    MixIntKernelType,
    MixHrcKernelType,
)
from smt.explainability_tools import (
    ShapFeatureImportanceDisplay, 
    ShapDisplay,
    PartialDependenceDisplay, 
    PDFeatureImportanceDisplay, 
    PDFeatureInteractionDisplay
)
from sklearn.metrics import mean_squared_error

import numpy as np
import pandas as pd
import time


In [2]:
# Objective function
def cantilever_deflection(x):
    norm_inertia_vals = [
        0.0833, 0.139, 0.380, 0.0796,
        0.133, 0.363, 0.0859, 0.136,
        0.360, 0.0922, 0.138, 0.369,
    ]
    
    I = int(x[0])
    L = x[1]
    S = x[2]
    norm_inertia = norm_inertia_vals[I]

    E = 200e9
    P = 50e3

    y = (P * L**3) / (3 * E * S**2 * norm_inertia)

    return y

def cantilever_deflection_vectorized(X):
    y = np.zeros(X.shape[0])
    for i in range(X.shape[0]):
        y[i] = cantilever_deflection(X[i, :])
    return y

In [3]:
%%time
"""Problem definition"""
# Design space
ds = DesignSpace([
    CategoricalVariable([
        "type_A", "type_B", "type_C", "type_D", 
        "type_E", "type_F", "type_G", "type_H", 
        "type_I", "type_J", "type_K", "type_L",
    ]),
    FloatVariable(10.0, 20.0),
    FloatVariable(1.0, 2.0),
])
categorical_feature_idx = [0]

f_obj = cantilever_deflection

# Create training and testing data
n_data = 200
n_train = int(0.8 * n_data)
n_test = n_data - n_train
xdoe, _ = ds.sample_valid_x(n_data)
y_doe = [f_obj(xdoe[i]) for i in range(len(xdoe))]

X_tr, y_tr = xdoe[:n_train, :], y_doe[:n_train]
X_te, y_te = xdoe[n_train:, :], y_doe[n_train:]
y_tr, y_te = np.array(y_tr), np.array(y_te)

# Name of the features
feature_names = [r'$I$', r'$L$', r'$S$']

# create mapping for the categories
categories_map = dict()
inverse_categories_map = dict()
for feature_idx in categorical_feature_idx:
    categories_map[feature_idx] = {
        i: value for i, value in enumerate(ds._design_variables[feature_idx].values)
    }
    inverse_categories_map[feature_idx] = {
        value: i for value, i in enumerate(ds._design_variables[feature_idx].values)
    }

CPU times: user 2.6 s, sys: 15.9 ms, total: 2.62 s
Wall time: 2.62 s


In [75]:
from itertools import product
from scipy import stats
from scipy.special import comb
import numpy as np

def get_reference_feature_values(x, is_categorical):
    # get reference values for each feature
    # if the feature is categorical/ordinal -> mode
    # else -> mean
    num_features = x.shape[1]
    reference_values = np.zeros(num_features)
    for feature_idx in range(num_features):
        if is_categorical[feature_idx] == 1:
            # mode = stats.mode(x[:, feature_idx], keepdims=False)[0]
            # reference_values[feature_idx] = mode
            reference_values[feature_idx] = np.random.choice(x[:, feature_idx])
        else:
            mean = np.mean(x[:, feature_idx])
            reference_values[feature_idx] = mean
    return reference_values

def create_mask_array(m):
    mask = np.array(list(product(range(2), repeat=m)))
    # remove mask where all elements are 0 / 1
    mask = mask[
        (~np.all(mask == 0, axis=1)) &
        (~np.all(mask == 1, axis=1))
    ]
    return mask


def calculate_weight(mask_row):
    m = len(mask_row)
    z = np.sum(mask_row)
    numerator = m - 1
    denominator = comb(m, z) * z * (m - z)
    weight = numerator / denominator
    return weight


def compute_shap_values(
    mask,
    s_full,
    weights,
    reference_values,
    model,
):
    y = model.predict_values(s_full)
    b0 = model.predict_values(reference_values.reshape(1, -1))
    y = y - b0

    w = np.diag(weights)

    b = np.dot(
        np.linalg.inv(np.dot(np.dot(mask.transpose(), w), mask)),
        np.dot(np.dot(mask.transpose(), w), y)
    )
    b = b.reshape(-1, )
    return b

In [155]:
instance = X_tr[0, :].copy()

instance = instance.reshape(1, -1)
mask = create_mask_array(instance.shape[1])
# mask_ = np.where(mask==0, -1, mask)
# mask = np.repeat(mask, 3, axis=0)
s_with_zero = mask * instance
# reference_values = get_reference_feature_values(X_tr, is_categorical)
reference_values_ = np.ones(mask.shape)
for i in range(len(reference_values_)):
    reference_values_[i, :] = get_reference_feature_values(X_tr, is_categorical)

s_full = (s_with_zero == 0) * reference_values_ + s_with_zero
weights = np.apply_along_axis(calculate_weight, 1, mask)


In [148]:
(s_with_zero == 0) * reference_values_

array([[ 1.        , 15.13224253,  0.        ],
       [ 1.        ,  0.        ,  1.51612486],
       [ 8.        ,  0.        ,  0.        ],
       [ 0.        , 15.13224253,  1.51612486],
       [ 0.        , 15.13224253,  0.        ],
       [ 0.        ,  0.        ,  1.51612486]])

In [149]:
s_with_zero

array([[ 0.        ,  0.        ,  1.05761935],
       [ 0.        , 19.05192784,  0.        ],
       [ 0.        , 19.05192784,  1.05761935],
       [ 3.        ,  0.        ,  0.        ],
       [ 3.        ,  0.        ,  1.05761935],
       [ 3.        , 19.05192784,  0.        ]])

In [156]:
instance = X_tr[0, :].copy()

instance = instance.reshape(1, -1)
mask = create_mask_array(instance.shape[1])

# reference_values = get_reference_feature_values(X_tr, is_categorical)
# reference_values_ = np.ones(mask.shape)
# for i in range(len(reference_values_)):
#     reference_values_[i, :] = get_reference_feature_values(X_tr, is_categorical)

s_ref = (mask == 0) * reference_values_
s_real = (mask == 1) * instance

s_full_2 = s_ref + s_real

# s_full = (s_with_zero == 0) * reference_values_ + s_with_zero
# weights = np.apply_along_axis(calculate_weight, 1, mask)


In [157]:
s_full == s_full_2

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [152]:
s_real

array([[ 0.        ,  0.        ,  1.05761935],
       [ 0.        , 19.05192784,  0.        ],
       [ 0.        , 19.05192784,  1.05761935],
       [ 3.        ,  0.        ,  0.        ],
       [ 3.        ,  0.        ,  1.05761935],
       [ 3.        , 19.05192784,  0.        ]])